# Convolutional Neural Networks

The convolutional network is perhaps the most straight forward graphical extension to the multi-layer perceptron (MLP) and one of the most popular classifier networks around. The network works by applying convolutions to each of the layers in an MLP restricting the amount of connections present between layers. First, let's remind ourselves that a convolution between two functions $f$ and $g$ is defined as:

$$ f \star g (t) = \int_{-\infty}^{\infty} f(t-s)g(s) ds .$$

The functions are meshed together by applying one as a filter over the other. If, for example, $g$ where a Gaussian it would blur $f$ by averaging it locally proportionally to the Gaussian kernel; the Gaussian decays at long distances and so far away elements of $f$ do not contribute to each other. [Insert Image Here]. The convolutional neural network applys this idea of filters or convolutions as operators to the input of the layer: each index is combined in some proportion to neighbouring indexes before being fed-forward through the activation function. Long range indexes are ignored and this greatly reduces the number of parameters per layer.

[Insert Image Here]

CNNs are arbitrarily general (just like MLPs) but they work best when the input information is arranged *topographically* i.e. there is a relationship between the amount of information between two indexes and the distance between them. An example of topologically ordered information is an image: if a given pixel is green then the pixels next to it are likely to green.

When defining a CNN we typically choose the filter size (in the dimensions of the input-output data) but we do not specify the weights of each filter; remember that the filter for every potential subset of indexes in a layer will have different weights. The weights are therefore learned. Let's create our own layer which performs a convolution on an image.

In [ ]:
mutable struct conv <: AbstractLayer
    W
    b
    f
    function conv(dimsin, dimsout, kernelsize, stride, activation)
        
        new(W, b, f)
    end
end

function feed_forward(layer::conv, data)
    for 
    
    